<a href="https://colab.research.google.com/github/ddah0329/3th_OUTTA_AI_BootCamp/blob/colab/%EC%9D%B4%EC%BB%A4%EB%A8%B8%EC%8A%A4_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_%EB%A6%AC%EB%B7%B0_%EB%A7%88%EB%8B%A4%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller
!pip install --upgrade selenium webdriver-manager


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [2]:
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.31.1


In [3]:
!pip show selenium

Name: selenium
Version: 4.23.1
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import pandas as pd
import time

In [10]:
def collect_reviews(product_url):
    # 메인 데이터프레임 초기화
    reviews_df = pd.DataFrame(columns=['제품명', '리뷰어', '리뷰어 정보', '별점', '한달이상사용', '재구매여부', '피부타입', '피부고민', '자극도', '리뷰내용'])

    # 드라이버 초기화
    firefox_options = FirefoxOptions()
    firefox_options.add_argument('--headless')  # 브라우저를 백그라운드로 실행
    driver = webdriver.Firefox(options=firefox_options)

    # 상품 페이지로 이동
    driver.get(product_url)

    # 페이지 로딩 대기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="reviewInfo"]/a')))

    # 상품명 크롤링 (PK)
    product_name_element = driver.find_element(By.CSS_SELECTOR, 'p.prd_name')
    product_name = product_name_element.text

    # 리뷰 버튼 클릭
    button = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a')
    button.click()

    # 페이지네이션 처리를 위해 10페이지까지 반복
    for page in range(10):
        # 리뷰 섹션 로딩 대기
        review_section = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review_cont')))
        driver.execute_script("arguments[0].scrollIntoView(true);", review_section)

        # 리뷰 요소들을 가져오기
        reviews = driver.find_elements(By.CSS_SELECTOR, "div.review_cont")
        reviewers = driver.find_elements(By.CSS_SELECTOR, "div.user.clrfix")

        for review, reviewer in zip(reviews, reviewers):
            new_row = {}  # 각 리뷰의 데이터를 저장할 임시 딕셔너리

            new_row['제품명'] = product_name
            new_row['리뷰어'] = reviewer.find_element(By.CSS_SELECTOR, 'p.info_user').text
            new_row['리뷰어 정보'] = [tag.text for tag in reviewer.find_elements(By.CSS_SELECTOR, 'p.tag')]
            new_row['별점'] = review.find_element(By.CSS_SELECTOR, 'span.review_point').text

            month_rebuy = [element.text for element in reviewer.find_elements(By.CSS_SELECTOR,'#gdasList > li:nth-child(2) > div.info > div > div')]
            new_row['한달이상사용'] = 1 if any('한달' in text for text in month_rebuy) else 0
            new_row['재구매여부'] = 1 if any('재구매' in text for text in month_rebuy) else 0

            new_row['피부타입'] = review.find_element(By.CSS_SELECTOR, 'div.poll_sample > dl:nth-child(1) > dd > span').text
            new_row['피부고민'] = review.find_element(By.CSS_SELECTOR, 'div.poll_sample > dl:nth-child(2) > dd > span').text
            new_row['자극도'] = review.find_element(By.CSS_SELECTOR, 'div.poll_sample > dl:nth-child(3) > dd > span').text
            new_row['리뷰내용'] = review.find_element(By.CSS_SELECTOR, 'div.txt_inner').text

            # 새로운 행을 DataFrame에 추가
            reviews_df = pd.concat([reviews_df, pd.DataFrame([new_row])], ignore_index=True)

        # 다음 페이지 버튼 클릭
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'a.next')
            next_button.click()
            time.sleep(2)  # 페이지가 로드될 시간을 조금 대기
        except:
            print(f"더 이상 페이지가 없습니다. 현재 {page + 1} 페이지에서 종료합니다.")
            break

    # 드라이버 종료
    driver.quit()

    return reviews_df

# 테스트 실행
collect_reviews("https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000201592&dispCatNo=1000001000100150001&trackingCd=Cat1000001000100150001_Small&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%ED%81%AC%EB%A6%BC_%EC%86%8C_%ED%81%AC%EB%A6%BC__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=6")

더 이상 페이지가 없습니다. 현재 10 페이지에서 종료합니다.


,제품명,리뷰어,리뷰어 정보,별점,한달이상사용,재구매여부,피부타입,피부고민,자극도,리뷰내용
0,헤브블루 살몬 케어링 센텔라 크림 100ml,뽀시리,[],5점만점에 5점,0,0,복합성에 좋아요,진정에 좋아요,자극없이 순해요,📌 연어라인 크림 사용한 지 3년 사용한 사람의 후기 글\n\n전 뭐가 올라오고 민...
1,헤브블루 살몬 케어링 센텔라 크림 100ml,김튜리,[],5점만점에 5점,0,1,건성에 좋아요,진정에 좋아요,자극없이 순해요,🩷1년6개월 넘게 사용한 내돈내산 찐 애정템 연어크림 후기🩷\n\n원인 모를 피부염...
2,헤브블루 살몬 케어링 센텔라 크림 100ml,꿀찌,[],5점만점에 5점,0,0,복합성에 좋아요,진정에 좋아요,자극없이 순해요,아 이거 진짜 쟁여템 요물템임 다써서 다시 산지 일주일도 안지났는데 최저가로 구매 ...
3,헤브블루 살몬 케어링 센텔라 크림 100ml,여쿨이나라다,[약건성 여름쿨톤 모공 민감성],5점만점에 5점,0,0,건성에 좋아요,진정에 좋아요,자극없이 순해요,"헤브블루는 원래부터 브랜드 자체에 애정이 있어서 쿠션,컨실러,스파츌러,클렌징오일5통..."
4,헤브블루 살몬 케어링 센텔라 크림 100ml,맠리,[복합성 웜톤 각질 모공],5점만점에 5점,0,0,복합성에 좋아요,진정에 좋아요,보통이에요,"클린 뷰티의 선두주자 헤브블루가 올영에 입점되어 너무 좋아요. 이 전에 연어 앰플,..."
...,...,...,...,...,...,...,...,...,...,...
95,헤브블루 살몬 케어링 센텔라 크림 100ml,1004sel****,[],5점만점에 5점,0,0,지성에 좋아요,보습에 좋아요,자극없이 순해요,피부타입 상관없이 다 넘넘 만족스러워서 이범에 또 대량구매 했어요ㅠㅠ 토너부터 크림...
96,헤브블루 살몬 케어링 센텔라 크림 100ml,jueun****,[],5점만점에 5점,0,0,복합성에 좋아요,보습에 좋아요,자극없이 순해요,이제 없어서는 안 되는 크림! 쫀쫀한데 얼굴에 다 흡수되고 피부결이 좋아지는 게 느...
97,헤브블루 살몬 케어링 센텔라 크림 100ml,fj****,[],5점만점에 5점,0,0,복합성에 좋아요,진정에 좋아요,자극없이 순해요,피부가 갑자기 뒤집어져서 연어크림으로 진정시키려고 주문했어요! 성분도 너무 좋고 순...
98,헤브블루 살몬 케어링 센텔라 크림 100ml,tour****,[],5점만점에 5점,0,0,복합성에 좋아요,진정에 좋아요,보통이에요,드디어 도착!!! 토너랑 같이 야무지게 사용해볼게요💓
